[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/CoDeF-colab/blob/main/CoDeF_colab.ipynb)

In [ ]:
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1

# !git clone --recursive https://github.com/nvlabs/tiny-cuda-nn
# %cd /content/tiny-cuda-nn
# !cmake . -B build
# !cmake --build build --config RelWithDebInfo -j
# !pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

!git clone -b dev https://github.com/camenduru/CoDeF
%cd /content/CoDeF

!pip install -q pytorch_lightning einops ipdb torch_optimizer kornia scikit-video
!pip install -q https://huggingface.co/camenduru/CoDeF/resolve/main/tinycudann-1.7-cp310-cp310-linux_x86_64.whl

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/all_sequences.zip -d /content/CoDeF/ckpts -o all_sequences.zip
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/beauty_0.ckpt -d /content/CoDeF/ckpts -o beauty_0.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/beauty_1.ckpt -d /content/CoDeF/ckpts -o beauty_1.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/lemon_hit.ckpt -d /content/CoDeF/ckpts -o lemon_hit.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/scene_0.ckpt -d /content/CoDeF/ckpts -o scene_0.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/CoDeF/resolve/main/white_smoke.ckpt -d /content/CoDeF/ckpts -o white_smoke.ckpt

!unzip /content/CoDeF/ckpts/all_sequences.zip -d /content/CoDeF/ckpts
!rm -rf /content/CoDeF/ckpts/__MACOSX

# !mkdir /content/CoDeF/ckpts/all_sequences/scene_0/base
# !cp /content/CoDeF/ckpts/scene_0.ckpt /content/CoDeF/ckpts/all_sequences/scene_0/base/scene_0.ckpt

In [ ]:
# @title Train (Step 1)

GPUS=0
NAME="test"
EXP_NAME="base"
ROOT_DIRECTORY=f"all_sequences/{NAME}/{NAME}"
MODEL_SAVE_PATH=f"ckpts/all_sequences/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/{NAME}.ckpt"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
FLOW_DIRECTORY=f"all_sequences/{NAME}/{NAME}_flow"

!python train.py --root_dir $ROOT_DIRECTORY \
                --model_save_path $MODEL_SAVE_PATH \
                --log_save_path $LOG_SAVE_PATH  \
                --gpus $GPUS \
                --encode_w --annealed \
                --config $CONFIG_DIRECTORY \
                --exp_name $EXP_NAME
                # --mask_dir $MASK_DIRECTORY
                # --flow_dir $FLOW_DIRECTORY

In [ ]:
# @title Generate Canonical Image (Step 2)

GPUS=0
NAME="test"
EXP_NAME="base"
ROOT_DIRECTORY=f"all_sequences/{NAME}/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/last.ckpt"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"

!python train.py  --test --encode_w \
                  --root_dir $ROOT_DIRECTORY \
                  --log_save_path $LOG_SAVE_PATH \
                  --mask_dir $MASK_DIRECTORY \
                  --weight_path $WEIGHT_PATH \
                  --gpus $GPUS \
                  --config $CONFIG_DIRECTORY \
                  --exp_name $EXP_NAME \
                  --save_deform False

In [ ]:
# @title Generate Video (Step 3)

GPUS=0
NAME="test"
EXP_NAME="base"
ROOT_DIRECTORY=f"all_sequences/{NAME}/{NAME}"
LOG_SAVE_PATH=f"logs/test_all_sequences/{NAME}"
MASK_DIRECTORY=f"all_sequences/{NAME}/{NAME}_masks_0 all_sequences/{NAME}/{NAME}_masks_1"
CONFIG_DIRECTORY=f"configs/{NAME}/{EXP_NAME}.yaml"
CANONICAL_DIR=f"ckpts/all_sequences/{NAME}/{EXP_NAME}_control"
WEIGHT_PATH=f"ckpts/all_sequences/{NAME}/{EXP_NAME}/last.ckpt"

!python train.py --test --encode_w \
                --root_dir $ROOT_DIRECTORY \
                --log_save_path $LOG_SAVE_PATH \
                --mask_dir $MASK_DIRECTORY \
                --weight_path $WEIGHT_PATH \
                --gpus $GPUS \
                --canonical_dir $CANONICAL_DIR \
                --config $CONFIG_DIRECTORY \
                --exp_name $EXP_NAME